In [4]:
!pip install -q pennylane

In [5]:
import numpy as np 
import pennylane as qml

In [ ]:
# N.2.1a - Gates on mixed states

def apply_gate_mixed(rho,U):
    U_dagger = np.conjugate(U.T)
    return U @ rho @ U_dagger

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_mixed(rho,U).round(2))


A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62+0.j   0.25+0.22j]
 [0.25-0.22j 0.38+0.j  ]]


In [7]:
# N.2.1b - Applying a gate with PennyLane
    
dev = qml.device("default.mixed", wires=1)

@qml.qnode(dev)
def apply_gate_circuit(rho,U):
    
    # Prepare a state with density operator rho
    qml.QubitDensityMatrix(rho, wires=[0])
    
    # Apply the unitary U
    qml.QubitUnitary(U, wires=[0])
    
    return qml.state()

U = qml.matrix(qml.RX(np.pi/3,0))
rho = np.array([[3/4,1/4],[1/4,1/4]])

print("A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:")
print(apply_gate_circuit(rho,U).round(2))


A pi/3 RX rotation applied on [[3/4,1/4],[1/4,1/4]] gives:
[[0.62+0.j   0.25+0.22j]
 [0.25-0.22j 0.38+0.j  ]]


In [ ]:
# Eigen projectors and eigenvalues.
def eigenprojectors(obs):
    eigenvalue, eigenvectors = np.linalg.eig(obs)
    projectors = [np.outer(p, np.conjugate(p)) for p in eigenvectors]
    return projectors

def outcome_probs(rho, B):
    eigen_projectors = eigenprojectors(B)
    prob_list = [np.trace(rho @ P) for P in eigen_projectors]  # Calculate probabilities
    return np.array(prob_list)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliY(0))

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(B))
print("Measurement probabilities {}".format(outcome_probs(rho,B).round(2)))

State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5+0.j 0.5+0.j]


In [9]:
# N.2.2c - Measuring projective operators
dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def mixed_probs_circuit(rho, obs):
    # Prepare the density matrix
    qml.QubitDensityMatrix(rho, wires=[0])
    
    return qml.probs(op=obs)

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliY(0)

print("State: [[3/4,0],[0,1/4]], Observable: {}".format(qml.matrix(B)))
print("Measurement probabilities {}".format(mixed_probs_circuit(rho,B))) 


State: [[3/4,0],[0,1/4]], Observable: [[ 0.+0.j -0.-1.j]
 [ 0.+1.j  0.+0.j]]
Measurement probabilities [0.5 0.5]


In [11]:
# N.2.3a - Calculating the expectation values
def mixed_expval(rho, B):
    eigenvalue, eigenvectors = np.linalg.eig(B)
    projectors = [np.outer(p, np.conjugate(p)) for p in eigenvectors]
    prob_list = [np.trace(rho @ P) for P in projectors]  # Calculate probabilities

    exp_val = np.sum([a*b for a,b in zip(prob_list, eigenvalue)])
    
    return exp_val # Return the expectation value

rho = np.array([[3/4,0],[0,1/4]])
B = qml.matrix(qml.PauliZ(0))

print("State: {}".format(rho))
print("Observable: {}".format(B))
print("Expectation value: {}".format(mixed_expval(rho,B)))

# N.2.3b - Returning expectation values

dev = qml.device('default.mixed', wires = 1)

@qml.qnode(dev)
def expval_circuit(rho,obs):
    qml.QubitDensityMatrix(rho, wires=[0])
    return qml.expval(B) # Return the expectation value

rho = np.array([[3/4,0],[0,1/4]])
B = qml.PauliZ(0)

print("State: {}".format(rho))
print("Observable: {}".format(qml.matrix(B)))
print("Expectation value: {}".format(expval_circuit(rho,B)))


State: [[0.75 0.  ]
 [0.   0.25]]
Observable: [[ 1  0]
 [ 0 -1]]
Expectation value: 0.5
State: [[0.75 0.  ]
 [0.   0.25]]
Observable: [[ 1  0]
 [ 0 -1]]
Expectation value: 0.5
